In [16]:
import json
from pathlib import Path
from urllib.parse import urlparse

from pydantic_ai import Agent

from knd.tools import crawl_url

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
def url_filter(url: str) -> bool:
    return urlparse(url).path.startswith("/job/")


main_url = "https://www.nhsjobs.com/job_list?JobSearch_q=&JobSearch_d=&JobSearch_g=&JobSearch_re=_POST&JobSearch_re_0=&JobSearch_re_1=-_-_-&JobSearch_re_2=-_-_--_-_-&JobSearch_Submit=Search&_tr=JobSearch&_ts=206601&_pg={page}&_pgid="
res = {}
for page in range(1, 5):
    url = main_url.format(page=page)
    res.update(await crawl_url(url=url, max_depth=1, max_links=200, url_filter=url_filter))
    # res[url] = [_url.replace("</", "/") for _url in res.values()]

[INIT].... → Crawl4AI 0.4.247
[FETCH]... ↓ https://www.nhsjobs.com/job_list?JobSearch_q=&JobS... | Status: True | Time: 0.00s
[COMPLETE] ● https://www.nhsjobs.com/job_list?JobSearch_q=&JobS... | Status: True | Total: 0.01s
[INIT].... → Crawl4AI 0.4.247
[FETCH]... ↓ https://www.nhsjobs.com/job_list?JobSearch_q=&JobS... | Status: True | Time: 0.00s
[COMPLETE] ● https://www.nhsjobs.com/job_list?JobSearch_q=&JobS... | Status: True | Total: 0.01s
[INIT].... → Crawl4AI 0.4.247
[FETCH]... ↓ https://www.nhsjobs.com/job_list?JobSearch_q=&JobS... | Status: True | Time: 5.62s
[SCRAPE].. ◆ Processed https://www.nhsjobs.com/job_list?JobSearch_q=&JobS... | Time: 65ms
[COMPLETE] ● https://www.nhsjobs.com/job_list?JobSearch_q=&JobS... | Status: True | Total: 5.70s
[INIT].... → Crawl4AI 0.4.247
[FETCH]... ↓ https://www.nhsjobs.com/job_list?JobSearch_q=&JobS... | Status: True | Time: 3.72s
[SCRAPE].. ◆ Processed https://www.nhsjobs.com/job_list?JobSearch_q=&JobS... | Time: 62ms
[COMPLETE] ● https://www.

In [10]:
Path("nhsjobs.json").write_text(json.dumps(res, indent=2))

124720

In [13]:
Path("nhsjobs.md").write_text("\n\n-----------------\n\n".join(res.values()).replace("</", "/"))


120657

In [14]:
jobs = Path("nhsjobs.json").read_text()

In [15]:
len(jobs.split())

8082

In [25]:
agent = Agent(name="jobs_agent", model="google-gla:gemini-2.0-flash-exp")
filtered_jobs = await agent.run(
    user_prompt=f"I only want jobs that are for my level. My level is 'Foundation Year 2'. Return the filtered jobs in a markdown format with the links intact. Remove the images. Just a numbered list of the jobs with the title, any description, and the link. At least 15 jobs.\n\nHere are the jobs:\n{jobs}."
)


INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-exp:generateContent "HTTP/1.1 200 OK"


In [26]:
Path("filtered_jobs_gemini.md").write_text(filtered_jobs.data)



12006